In [1]:
from proj1_helpers import *
import numpy as np

In [2]:
## Load data
y_train, x_train, idx_train = load_csv_data("train.csv")
y_test, x_test, idx_test = load_csv_data("test.csv")

In [ ]:
def split_data(x_train):
    tx_der = x_train[:, :14]
    tx_pri = x_train[:, 13:]
    return tx_der, tx_pri

In [ ]:
np.corrcoef(x_train)


## helper function

In [3]:
## Strandardize
def standardize(x):
    mean_x = np.mean(x)
    x = x - mean_x
    std_x = np.std(x)
    x = x / std_x
    return x, mean_x, std_x

In [4]:
## Batch_iter
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]

In [5]:
## Build k_indices (Ridge Regression)
def build_k_indices(y, k_fold, seed):
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

## Transform Data

In [6]:
# remove -999
x_train[x_train == -999] = 0
x_test[x_test == -999] = 0
# Standardize

tx_train, tx_trmean, tx_trstd = standardize(x_train)
tx_test, tx_temena, tx_testd = standardize(x_test)

# ## split_data into der and pri
# tx_train_der, tx_train_pri = split_data(tx_train)
# tx_test_der, tx_test_pri = split_data(tx_test)


## Regression

### Least Squares GD

In [ ]:
def compute_gradient(y, tx, w):
    """Compute the gradient."""
    err = y - tx.dot(w)
    grad = -tx.T.dot(err) / len(err)
    return grad, err

def calculate_mse(y, tx, w):
    """mean square error"""
    err = y - tx.dot(w)
    return 1/2*np.mean(err**2)


In [ ]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):

    threshold = 1e-8

    losses = []
    w = initial_w

    for i in range(max_iters):
        # compute loss, gradient
        grad, err = compute_gradient(y, tx, w)
        loss = calculate_mse(y, tx, w)
        # gradient w by descent update
        w = w - gamma * grad
        # store w and loss
        losses.append(loss)
        
        print("GD({bi}/{ti}): loss={l}".format(
              bi=i, ti=max_iters - 1, l=loss))  
        
        # converge criteria
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    return losses[-1], w

In [ ]:
## training by least_squares_GD
max_iters = 10000
gamma = 0.01
initial_w = np.zeros(tx_train_der.shape[1])
lgd_loss, lgd_w = least_squares_GD(y_train, tx_train_der, initial_w, max_iters, gamma) 

In [ ]:
lgd_w

In [ ]:
lgd_loss

### Least Squares SGD

In [ ]:
def compute_stoch_gradient(y, tx, w):
    e = y - tx.dot(w)
    gradient = -tx.T.dot(e) / len(e)
    return gradient, e

In [ ]:
def least_squares_SGD(y, tx, initial_w, max_iters=1000, gamma=0.01):
    threshold = 1e-8
    losses = []
    w = initial_w

    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, 1, 1):
            
            grad, err = compute_stoch_gradient(y_batch, tx_batch, w)
            w = w - gamma * grad
            
            loss = calculate_mse(y, tx, w)
            losses.append(loss)
        
        print("SGD({bi}/{ti}): loss={l}".format(
              bi=n_iter, ti=max_iters - 1, l=loss))  
        
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break

    return losses[-1], w

In [ ]:
## training by least_squares_SGD
max_iters = 10000
gamma = 0.001
initial_w = np.zeros(tx_train.shape[1])
lsgd_loss, lsgd_w = least_squares_SGD(y_train, tx_train, initial_w, max_iters, gamma) 

In [ ]:
lsgd_w

In [ ]:
lsgd_loss

### Ridge Regression

In [ ]:
def ridge_regression(y, tx, lambda_):
    Xt = tx.T
    term1 = Xt.dot(tx) + 2 * tx.shape[0] * lambda_ * np.identity(tx.shape[1])
    weights = np.linalg.inv(term1).dot(Xt).dot(y)

    loss = compute_loss(y, tx, weights)
    return loss, weights

### Least Squares

In [ ]:
# Least squares
def least_squares(y, tx):
    
    """least squares regression using normal equations"""
    
    a = tx.T.dot(tx)
    b = tx.T.dot(y)
    w = np.linalg.solve(a, b)
    loss = calculate_mse(y, tx, w)
    
    return loss, w


In [ ]:
ls_loss, ls_w = least_squares(y_train, tx_train_pri)

In [ ]:
ls_loss

In [ ]:
ls_w

In [ ]:
ls_loss

### Logistic Regression (SGD)

In [17]:
def box_cox(x):
    if (x == 0):
        

def sigmoid(x):
    return np.exp(-np.logaddexp(0, -x))

def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    sig = sigmoid(tx.dot(w))
    gradient = tx.T.dot(sig - y)
    return gradient

def calculate_negative_log_likelihood(y, tx, w):
    """compute the cost by negative log likelihood."""
    epsilon = 1e-12
    sig = sigmoid(tx.dot(w))
    loss = y.T.dot(np.log(sig + epsilon )) + (1 - y).T.dot(np.log(1 - sig + epsilon))

    return np.squeeze(-loss)

def learning_by_gradient_descent(y, tx, w, gamma):

    loss = calculate_negative_log_likelihood(y, tx, w)
    gradient = calculate_gradient(y, tx, w)
    w = w - gamma * gradient
    return loss, w

In [18]:
def logistic_regression_SGD(y, tx, initial_w, max_iters, gamma):
    """Logistic regression using stochastic SGD"""

    threshold = 1e-8
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=1, num_batches=1):
            # compute a stochastic gradient and loss
            grad = calculate_gradient(y_batch, tx_batch, w)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # calculate loss
            loss = calculate_negative_log_likelihood(y, tx, w)
            # store w and loss

            losses.append(loss)

        print("LR_SGD({bi}/{ti}): loss={l}".format(
              bi=n_iter, ti=max_iters - 1, l=loss))
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
        
    return losses[-1], w

In [19]:
## training by logistci_regression
max_iters = 10000
gamma = 0.001
initial_w = np.zeros(tx_train.shape[1])
lrsgd_loss, lrsgd_w = logistic_regression_SGD(y_train, tx_train, initial_w, max_iters, gamma) 

LR_SGD(0/4999): loss=174002.69733087323
LR_SGD(1/4999): loss=175029.6066102339
LR_SGD(2/4999): loss=173040.3773916068
LR_SGD(3/4999): loss=171336.86583928028
LR_SGD(4/4999): loss=172724.62706090487
LR_SGD(5/4999): loss=170040.14680903545
LR_SGD(6/4999): loss=170925.7633430998
LR_SGD(7/4999): loss=168487.04627542468
LR_SGD(8/4999): loss=169307.82849522715
LR_SGD(9/4999): loss=165621.49757442775
LR_SGD(10/4999): loss=164258.39389870805
LR_SGD(11/4999): loss=164825.39624994906
LR_SGD(12/4999): loss=165839.16480029433
LR_SGD(13/4999): loss=163587.78288350243
LR_SGD(14/4999): loss=161131.23483095237
LR_SGD(15/4999): loss=162103.74784343078
LR_SGD(16/4999): loss=159010.15820905886
LR_SGD(17/4999): loss=156085.75221938777
LR_SGD(18/4999): loss=154679.77710999548
LR_SGD(19/4999): loss=148374.58746120255
LR_SGD(20/4999): loss=149458.66604861311
LR_SGD(21/4999): loss=150314.593860671
LR_SGD(22/4999): loss=148432.43986384541
LR_SGD(23/4999): loss=142017.23144045443
LR_SGD(24/4999): loss=138205.21

LR_SGD(200/4999): loss=-24489.391433863682
LR_SGD(201/4999): loss=-25735.300435974146
LR_SGD(202/4999): loss=-24901.302326325298
LR_SGD(203/4999): loss=-25907.06236913039
LR_SGD(204/4999): loss=-26937.628256793134
LR_SGD(205/4999): loss=-27437.01909461929
LR_SGD(206/4999): loss=-28597.863179760607
LR_SGD(207/4999): loss=-29510.759906984007
LR_SGD(208/4999): loss=-30781.277798096635
LR_SGD(209/4999): loss=-31793.271563045753
LR_SGD(210/4999): loss=-32546.352130045998
LR_SGD(211/4999): loss=-30852.557377446123
LR_SGD(212/4999): loss=-31717.54804566645
LR_SGD(213/4999): loss=-31094.93507878638
LR_SGD(214/4999): loss=-29692.79120559407
LR_SGD(215/4999): loss=-28715.141154749814
LR_SGD(216/4999): loss=-29620.33173780926
LR_SGD(217/4999): loss=-31384.052994943777
LR_SGD(218/4999): loss=-30037.948107414573
LR_SGD(219/4999): loss=-31358.244855960234
LR_SGD(220/4999): loss=-33347.77946870777
LR_SGD(221/4999): loss=-34532.445656080934
LR_SGD(222/4999): loss=-33357.01028628745
LR_SGD(223/4999): l

LR_SGD(397/4999): loss=-109976.97472180505
LR_SGD(398/4999): loss=-111161.3880173828
LR_SGD(399/4999): loss=-112169.80825060465
LR_SGD(400/4999): loss=-111059.15639983566
LR_SGD(401/4999): loss=-110465.46140706068
LR_SGD(402/4999): loss=-111298.4637627971
LR_SGD(403/4999): loss=-112072.34329307705
LR_SGD(404/4999): loss=-112875.26577852179
LR_SGD(405/4999): loss=-113406.9490788104
LR_SGD(406/4999): loss=-114065.43864862222
LR_SGD(407/4999): loss=-115009.31024431848
LR_SGD(408/4999): loss=-116159.06615957302
LR_SGD(409/4999): loss=-115525.52627698127
LR_SGD(410/4999): loss=-116554.52582567587
LR_SGD(411/4999): loss=-117773.87949460628
LR_SGD(412/4999): loss=-116719.3237810052
LR_SGD(413/4999): loss=-117365.17196018319
LR_SGD(414/4999): loss=-116693.2680297381
LR_SGD(415/4999): loss=-116164.88909636682
LR_SGD(416/4999): loss=-115708.88385831853
LR_SGD(417/4999): loss=-116840.7338320939
LR_SGD(418/4999): loss=-117627.14350662436
LR_SGD(419/4999): loss=-117082.31807521418
LR_SGD(420/4999):

LR_SGD(590/4999): loss=-155672.92323972925
LR_SGD(591/4999): loss=-155116.7479843762
LR_SGD(592/4999): loss=-154472.3694757152
LR_SGD(593/4999): loss=-153907.6812679806
LR_SGD(594/4999): loss=-152975.90816398704
LR_SGD(595/4999): loss=-153900.47389117494
LR_SGD(596/4999): loss=-154737.68665839895
LR_SGD(597/4999): loss=-156138.77808860037
LR_SGD(598/4999): loss=-157077.4326179462
LR_SGD(599/4999): loss=-157996.60329276772
LR_SGD(600/4999): loss=-157094.54866245342
LR_SGD(601/4999): loss=-158738.19023246592
LR_SGD(602/4999): loss=-159559.68284460457
LR_SGD(603/4999): loss=-160142.31048570093
LR_SGD(604/4999): loss=-160878.4281962764
LR_SGD(605/4999): loss=-161419.81617173378
LR_SGD(606/4999): loss=-162908.0773580874
LR_SGD(607/4999): loss=-163673.40542187775
LR_SGD(608/4999): loss=-164491.765861215
LR_SGD(609/4999): loss=-162730.84742407413
LR_SGD(610/4999): loss=-163555.1579279533
LR_SGD(611/4999): loss=-164632.8045656521
LR_SGD(612/4999): loss=-165482.65804009
LR_SGD(613/4999): loss=-

LR_SGD(783/4999): loss=-235684.70197219058
LR_SGD(784/4999): loss=-236578.25312323374
LR_SGD(785/4999): loss=-237228.25544212316
LR_SGD(786/4999): loss=-236462.3907069865
LR_SGD(787/4999): loss=-237124.53279209856
LR_SGD(788/4999): loss=-236447.04637289172
LR_SGD(789/4999): loss=-237246.93453702654
LR_SGD(790/4999): loss=-237858.00662062853
LR_SGD(791/4999): loss=-237210.26558373915
LR_SGD(792/4999): loss=-236951.7014948195
LR_SGD(793/4999): loss=-237523.6902052235
LR_SGD(794/4999): loss=-238060.79482568474
LR_SGD(795/4999): loss=-238592.70221458958
LR_SGD(796/4999): loss=-239254.12598709462
LR_SGD(797/4999): loss=-239932.97814820806
LR_SGD(798/4999): loss=-240447.5851186408
LR_SGD(799/4999): loss=-241004.46672894817
LR_SGD(800/4999): loss=-241551.15277969808
LR_SGD(801/4999): loss=-240606.83579137502
LR_SGD(802/4999): loss=-240009.46694782583
LR_SGD(803/4999): loss=-240771.37532097896
LR_SGD(804/4999): loss=-240235.35616639355
LR_SGD(805/4999): loss=-239567.17660896998
LR_SGD(806/4999

...

LR_SGD(4062/4999): loss=-917482.2282991776
LR_SGD(4063/4999): loss=-917243.2535493075
LR_SGD(4064/4999): loss=-916840.5310326657
LR_SGD(4065/4999): loss=-917397.2102616575
LR_SGD(4066/4999): loss=-916964.5972559137
LR_SGD(4067/4999): loss=-917575.8522756265
LR_SGD(4068/4999): loss=-918253.7322658004
LR_SGD(4069/4999): loss=-918167.4572411593
LR_SGD(4070/4999): loss=-918666.8646430235
LR_SGD(4071/4999): loss=-919381.0721051791
LR_SGD(4072/4999): loss=-919879.6271382523
LR_SGD(4073/4999): loss=-919932.4067611768
LR_SGD(4074/4999): loss=-920370.4615941002
LR_SGD(4075/4999): loss=-919937.2656415058
LR_SGD(4076/4999): loss=-920381.685878175
LR_SGD(4077/4999): loss=-920630.6661927304
LR_SGD(4078/4999): loss=-920671.4609474355
LR_SGD(4079/4999): loss=-920236.7793877814
LR_SGD(4080/4999): loss=-919777.3325855912
LR_SGD(4081/4999): loss=-919092.9494027357
LR_SGD(4082/4999): loss=-918741.1871861678
LR_SGD(4083/4999): loss=-919559.5525306045
LR_SGD(4084/4999): loss=-919008.4642976873
LR_SGD(4085/

LR_SGD(4254/4999): loss=-952244.4185101133
LR_SGD(4255/4999): loss=-952684.4313363056
LR_SGD(4256/4999): loss=-953267.2712969803
LR_SGD(4257/4999): loss=-953271.3219130577
LR_SGD(4258/4999): loss=-953761.4688953386
LR_SGD(4259/4999): loss=-954611.3493928338
LR_SGD(4260/4999): loss=-955078.2085860481
LR_SGD(4261/4999): loss=-954499.5614328238
LR_SGD(4262/4999): loss=-954055.0384436961
LR_SGD(4263/4999): loss=-954491.0231271513
LR_SGD(4264/4999): loss=-955066.4710813018
LR_SGD(4265/4999): loss=-954431.1713224271
LR_SGD(4266/4999): loss=-954081.1609706423
LR_SGD(4267/4999): loss=-953398.6021227902
LR_SGD(4268/4999): loss=-953886.9037017587
LR_SGD(4269/4999): loss=-953452.6681560245
LR_SGD(4270/4999): loss=-954213.8115076763
LR_SGD(4271/4999): loss=-954616.434079368
LR_SGD(4272/4999): loss=-955205.4714925308
LR_SGD(4273/4999): loss=-954402.9179491487
LR_SGD(4274/4999): loss=-954787.2905277874
LR_SGD(4275/4999): loss=-954223.9620123265
LR_SGD(4276/4999): loss=-954710.2005229527
LR_SGD(4277/

LR_SGD(4448/4999): loss=-979780.5636456341
LR_SGD(4449/4999): loss=-980512.854095316
LR_SGD(4450/4999): loss=-980889.4794135503
LR_SGD(4451/4999): loss=-981422.9341067112
LR_SGD(4452/4999): loss=-981425.3797242916
LR_SGD(4453/4999): loss=-982251.62805579
LR_SGD(4454/4999): loss=-982652.2076235336
LR_SGD(4455/4999): loss=-983077.8029357961
LR_SGD(4456/4999): loss=-983507.870225423
LR_SGD(4457/4999): loss=-984100.1278910256
LR_SGD(4458/4999): loss=-984734.144552678
LR_SGD(4459/4999): loss=-984171.5165964306
LR_SGD(4460/4999): loss=-983696.5801615587
LR_SGD(4461/4999): loss=-984238.8764598272
LR_SGD(4462/4999): loss=-985643.8255495349
LR_SGD(4463/4999): loss=-986141.2472002007
LR_SGD(4464/4999): loss=-986816.1964411839
LR_SGD(4465/4999): loss=-987523.3864403397
LR_SGD(4466/4999): loss=-987751.0366632864
LR_SGD(4467/4999): loss=-987246.2918968423
LR_SGD(4468/4999): loss=-987659.423515182
LR_SGD(4469/4999): loss=-988165.739630304
LR_SGD(4470/4999): loss=-989035.2219296581
LR_SGD(4471/4999):

LR_SGD(4641/4999): loss=-1008951.8335789791
LR_SGD(4642/4999): loss=-1009525.9213244566
LR_SGD(4643/4999): loss=-1010256.4622300351
LR_SGD(4644/4999): loss=-1010768.5555527476
LR_SGD(4645/4999): loss=-1011137.2682151933
LR_SGD(4646/4999): loss=-1010554.7788771207
LR_SGD(4647/4999): loss=-1011793.6476629768
LR_SGD(4648/4999): loss=-1011195.1727533508
LR_SGD(4649/4999): loss=-1010676.7727490446
LR_SGD(4650/4999): loss=-1010205.3797764657
LR_SGD(4651/4999): loss=-1010447.8067614823
LR_SGD(4652/4999): loss=-1010832.35142667
LR_SGD(4653/4999): loss=-1010187.2266158556
LR_SGD(4654/4999): loss=-1009602.7719050428
LR_SGD(4655/4999): loss=-1010171.9998343978
LR_SGD(4656/4999): loss=-1010584.7659645743
LR_SGD(4657/4999): loss=-1009975.1659855373
LR_SGD(4658/4999): loss=-1010565.4836381614
LR_SGD(4659/4999): loss=-1011267.5636214361
LR_SGD(4660/4999): loss=-1010394.7093246614
LR_SGD(4661/4999): loss=-1009939.1354330528
LR_SGD(4662/4999): loss=-1010492.1444170695
LR_SGD(4663/4999): loss=-1011048.1

LR_SGD(4829/4999): loss=-1038704.1268341827
LR_SGD(4830/4999): loss=-1038245.3239708432
LR_SGD(4831/4999): loss=-1037721.2030346827
LR_SGD(4832/4999): loss=-1037997.781304884
LR_SGD(4833/4999): loss=-1037288.2761313325
LR_SGD(4834/4999): loss=-1037758.4817434745
LR_SGD(4835/4999): loss=-1038206.819197384
LR_SGD(4836/4999): loss=-1038602.3452980428
LR_SGD(4837/4999): loss=-1039095.5802339448
LR_SGD(4838/4999): loss=-1039794.8978958213
LR_SGD(4839/4999): loss=-1040258.7956456756
LR_SGD(4840/4999): loss=-1041262.9463206516
LR_SGD(4841/4999): loss=-1041924.3946813942
LR_SGD(4842/4999): loss=-1042746.0850303433
LR_SGD(4843/4999): loss=-1043176.3645820749
LR_SGD(4844/4999): loss=-1043483.5449820863
LR_SGD(4845/4999): loss=-1042733.3399589525
LR_SGD(4846/4999): loss=-1042081.0986994827
LR_SGD(4847/4999): loss=-1042866.3751273626
LR_SGD(4848/4999): loss=-1043357.0254270534
LR_SGD(4849/4999): loss=-1042907.4294505067
LR_SGD(4850/4999): loss=-1043664.0470052388
LR_SGD(4851/4999): loss=-1044309.1

In [20]:
lrsgd_w

array([-0.80445368, -0.99812596, -1.0321358 ,  0.18005652,  0.74885115,
        1.28938028,  0.73451031,  0.69746881,  0.29480696, -1.67685883,
        0.70716195,  0.76729255,  0.74825969,  0.23677741,  0.74669472,
        0.74740555, -0.28458231,  0.74739993,  0.74569285, -0.1336589 ,
        0.74927495, -2.72017334,  0.73347159,  0.22603133,  0.74600548,
        0.74638318,  0.55940817,  0.74785392,  0.74601221, -0.13437263])

In [13]:
lrsgd_loss

-859519.2005315799

In [21]:
lrsgd_loss

-1064692.0518033062

###  Regualarize Logistic Regression

In [35]:

def penalized_logistic_regression(y, tx, w, lambda_):
    
    loss = calculate_negative_log_likelihood(y, tx, w) + lambda_ * np.squeeze(w.T.dot(w))
    gradient = calculate_gradient(y, tx, w) + 2 * lambda_ * w

    return loss, gradient

def reg_logistic_regression(y, tx, lambda_ , initial_w, max_iters, gamma):

    threshold = 1e-8
    losses = []
    w = initial_w
    
    # start the logistic regression
    for n_iter in range(max_iters):
        # get loss and update w.
        loss, gradient = penalized_logistic_regression(y, tx, w, lambda_)
        w = w - gamma * gradient

        # converge criteria
        losses.append(loss)
        print("LR_SGD({bi}/{ti}): loss={l}, norm of {w}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w= np.inner(w,w)))
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
            
    return losses[-1], w

In [37]:
## training by regularize logistci_regression

max_iters = 10000
gamma = 0.001
lambda_ = 0.1
initial_w = np.zeros(tx_train.shape[1])

rlrsgd_loss, rlrsgd_w = reg_logistic_regression(y_train, tx_train, lambda_, initial_w, max_iters, gamma) 

LR_SGD(0/9999): loss=173286.79513500613, norm of 359675.8399892417
LR_SGD(1/9999): loss=-1956519.1641846022, norm of 648240.6502098794
LR_SGD(2/9999): loss=-1931067.2114647499, norm of 1041795.6837580389
LR_SGD(3/9999): loss=-1911980.4348648079, norm of 1535636.9567420378
LR_SGD(4/9999): loss=-1875557.570681922, norm of 2127326.028329721
LR_SGD(5/9999): loss=-1821981.129957403, norm of 2816207.907059348
LR_SGD(6/9999): loss=-1756151.7730197443, norm of 3602050.93624609
LR_SGD(7/9999): loss=-1678535.5376062843, norm of 4484738.868256625
LR_SGD(8/9999): loss=-1591284.8667556585, norm of 5464190.157558347
LR_SGD(9/9999): loss=-1494094.31244508, norm of 6540334.098856934
LR_SGD(10/9999): loss=-1386996.5037327711, norm of 7713106.919103111
LR_SGD(11/9999): loss=-1270104.6293714917, norm of 8982448.11321969
LR_SGD(12/9999): loss=-1143450.2118311636, norm of 10348298.264770927
LR_SGD(13/9999): loss=-1007005.518884633, norm of 11810598.455969466
LR_SGD(14/9999): loss=-860931.7703172709, norm o

LR_SGD(125/9999): loss=73982216.91175236, norm of 772152452.9360347
LR_SGD(126/9999): loss=75172610.67859533, norm of 784146442.9139947
LR_SGD(127/9999): loss=76372010.86290525, norm of 796230443.703159
LR_SGD(128/9999): loss=77580411.84114912, norm of 808404400.3412186
LR_SGD(129/9999): loss=78797808.09982772, norm of 820668257.8916392
LR_SGD(130/9999): loss=80024194.25750393, norm of 833021961.4436514
LR_SGD(131/9999): loss=81259564.92809539, norm of 845465456.112239
LR_SGD(132/9999): loss=82503914.6727725, norm of 857998687.0381278
LR_SGD(133/9999): loss=83757238.02320956, norm of 870621599.387775
LR_SGD(134/9999): loss=85019529.5014877, norm of 883334138.3533579
LR_SGD(135/9999): loss=86290783.6283936, norm of 896136249.1527628
LR_SGD(136/9999): loss=87570994.92627898, norm of 909027877.0295738
LR_SGD(137/9999): loss=88860157.91996713, norm of 922008967.2530621
LR_SGD(138/9999): loss=90158267.1370003, norm of 935079465.118174
LR_SGD(139/9999): loss=91465317.10766146, norm of 948239

LR_SGD(249/9999): loss=288593702.4656512, norm of 2929023850.8848777
LR_SGD(250/9999): loss=290859766.8539617, norm of 2951767929.4241467
LR_SGD(251/9999): loss=293134175.2906803, norm of 2974595396.6173983
LR_SGD(252/9999): loss=295416922.58610415, norm of 2997506200.6154337
LR_SGD(253/9999): loss=297708003.55239207, norm of 3020500289.593486
LR_SGD(254/9999): loss=300007413.00345415, norm of 3043577611.7512193
LR_SGD(255/9999): loss=302315145.75487256, norm of 3066738115.3127117
LR_SGD(256/9999): loss=304631196.6238914, norm of 3089981748.526447
LR_SGD(257/9999): loss=306955560.4295387, norm of 3113308459.6653037
LR_SGD(258/9999): loss=309288231.99294513, norm of 3136718197.0265465
LR_SGD(259/9999): loss=311629206.13790494, norm of 3160210908.931811
LR_SGD(260/9999): loss=313978477.69164604, norm of 3183786543.7270994
LR_SGD(261/9999): loss=316336041.48565656, norm of 3207445049.7827654
LR_SGD(262/9999): loss=318701892.356304, norm of 3231186375.493505
LR_SGD(263/9999): loss=32107602

LR_SGD(372/9999): loss=628377152.8952876, norm of 6336739525.949954
LR_SGD(373/9999): loss=631631344.8709617, norm of 6369358684.989763
LR_SGD(374/9999): loss=634893260.8819728, norm of 6402055035.439564
LR_SGD(375/9999): loss=638162896.0339043, norm of 6434828528.378324
LR_SGD(376/9999): loss=641440245.4346582, norm of 6467679114.908192
LR_SGD(377/9999): loss=644725304.194454, norm of 6500606746.154478
LR_SGD(378/9999): loss=648018067.425827, norm of 6533611373.26565
LR_SGD(379/9999): loss=651318530.2436278, norm of 6566692947.413324
LR_SGD(380/9999): loss=654626687.7650219, norm of 6599851419.79225
LR_SGD(381/9999): loss=657942535.1094873, norm of 6633086741.620305
LR_SGD(382/9999): loss=661266067.3988152, norm of 6666398864.138478
LR_SGD(383/9999): loss=664597279.7571069, norm of 6699787738.610872
LR_SGD(384/9999): loss=667936167.3107753, norm of 6733253316.3246765
LR_SGD(385/9999): loss=671282725.1885418, norm of 6766795548.590177
LR_SGD(386/9999): loss=674636948.5214368, norm of 6

KeyboardInterrupt: 

In [ ]:
rlrsgd_w

In [ ]:
rlrsgd_loss

## Prediction 


In [25]:
# y_pred
y_pred  = predict_labels(lrsgd_w, tx_test)

In [26]:
# Output to csv
create_csv_submission(idx_test, y_pred, "lrsgd_00000001")